In [7]:
import datetime
from pyspark.sql import DataFrame, Row
from pyspark.sql.types import *
from pyspark.sql.functions import unix_timestamp, from_unixtime, col
from pyspark.sql import functions as F
import sys
from hops import hdfs as hdfs
print(hdfs.project_path())

hdfs://rpc.namenode.service.consul:8020/Projects/pit/

In [26]:
fg1_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f1", StringType(), True)    
])

fg1=spark.read.csv("hdfs:///Projects/" + hdfs.project_name() + "/Resources/5-4-1-out.csv", header=True, schema=fg1_schema)
fg1=fg1.sort(col("id"),col("ts"))
fg1.show()

+---+----+-----+
| id|  ts|   f1|
+---+----+-----+
|  1|1001| f1-1|
|  1|1002| f1-2|
|  1|1003| f1-3|
|  1|1004| f1-4|
|  2|1001| f1-5|
|  2|1002| f1-6|
|  2|1003| f1-7|
|  2|1004| f1-8|
|  3|1001| f1-9|
|  3|1002|f1-10|
|  3|1003|f1-11|
|  3|1004|f1-12|
|  4|1001|f1-13|
|  4|1002|f1-14|
|  4|1003|f1-15|
|  4|1004|f1-16|
|  5|1001|f1-17|
|  5|1002|f1-18|
|  5|1003|f1-19|
|  5|1004|f1-20|
+---+----+-----+

In [18]:
fg2_schema = StructType([
  StructField("id", IntegerType(), True),
  StructField("ts", IntegerType(), True),
  StructField("f2", StringType(), True)    
])

fg2=spark.read.csv("hdfs:///Projects/" + hdfs.project_name() + "/Resources/5-4-2-out.csv", header=True, schema=fg2_schema)
fg2.show()

+---+----+-----+
| id|  ts|   f2|
+---+----+-----+
|  1|1002| f2-1|
|  1|1004| f2-2|
|  2|1002| f2-3|
|  2|1004| f2-4|
|  3|1002| f2-5|
|  3|1004| f2-6|
|  4|1002| f2-7|
|  4|1004| f2-8|
|  5|1002| f2-9|
|  5|1004|f2-10|
+---+----+-----+

In [39]:
joined=fg1.join(fg2, (fg1.id == fg2.id) & (fg1.ts == fg2.ts), how="full_outer").orderBy(fg1.id, fg1.ts)
joined.show()

+---+----+-----+----+----+-----+
| id|  ts|   f1|  id|  ts|   f2|
+---+----+-----+----+----+-----+
|  1|1001| f1-1|null|null| null|
|  1|1002| f1-2|   1|1002| f2-1|
|  1|1003| f1-3|null|null| null|
|  1|1004| f1-4|   1|1004| f2-2|
|  2|1001| f1-5|null|null| null|
|  2|1002| f1-6|   2|1002| f2-3|
|  2|1003| f1-7|null|null| null|
|  2|1004| f1-8|   2|1004| f2-4|
|  3|1001| f1-9|null|null| null|
|  3|1002|f1-10|   3|1002| f2-5|
|  3|1003|f1-11|null|null| null|
|  3|1004|f1-12|   3|1004| f2-6|
|  4|1001|f1-13|null|null| null|
|  4|1002|f1-14|   4|1002| f2-7|
|  4|1003|f1-15|null|null| null|
|  4|1004|f1-16|   4|1004| f2-8|
|  5|1001|f1-17|null|null| null|
|  5|1002|f1-18|   5|1002| f2-9|
|  5|1003|f1-19|null|null| null|
|  5|1004|f1-20|   5|1004|f2-10|
+---+----+-----+----+----+-----+

In [40]:
joined=fg1.join(fg2, (fg1.id == fg2.id) & (fg1.ts == fg2.ts), how="left_outer").orderBy(fg1.id, fg1.ts)
joined.show()

+---+----+-----+----+----+-----+
| id|  ts|   f1|  id|  ts|   f2|
+---+----+-----+----+----+-----+
|  1|1001| f1-1|null|null| null|
|  1|1002| f1-2|   1|1002| f2-1|
|  1|1003| f1-3|null|null| null|
|  1|1004| f1-4|   1|1004| f2-2|
|  2|1001| f1-5|null|null| null|
|  2|1002| f1-6|   2|1002| f2-3|
|  2|1003| f1-7|null|null| null|
|  2|1004| f1-8|   2|1004| f2-4|
|  3|1001| f1-9|null|null| null|
|  3|1002|f1-10|   3|1002| f2-5|
|  3|1003|f1-11|null|null| null|
|  3|1004|f1-12|   3|1004| f2-6|
|  4|1001|f1-13|null|null| null|
|  4|1002|f1-14|   4|1002| f2-7|
|  4|1003|f1-15|null|null| null|
|  4|1004|f1-16|   4|1004| f2-8|
|  5|1001|f1-17|null|null| null|
|  5|1002|f1-18|   5|1002| f2-9|
|  5|1003|f1-19|null|null| null|
|  5|1004|f1-20|   5|1004|f2-10|
+---+----+-----+----+----+-----+

In [48]:
fg1a=fg1.withColumnRenamed("id", "id_1").withColumnRenamed("ts", "ts_1")
joined=fg1a.join(fg2, (fg1a.id_1 == fg2.id) & (fg1a.ts_1 == fg2.ts), how="left_outer").drop(fg2.id).orderBy(fg1a.id_1, fg1a.ts_1)
joined.show()

+----+----+-----+----+-----+
|id_1|ts_1|   f1|  ts|   f2|
+----+----+-----+----+-----+
|   1|1001| f1-1|null| null|
|   1|1002| f1-2|1002| f2-1|
|   1|1003| f1-3|null| null|
|   1|1004| f1-4|1004| f2-2|
|   2|1001| f1-5|null| null|
|   2|1002| f1-6|1002| f2-3|
|   2|1003| f1-7|null| null|
|   2|1004| f1-8|1004| f2-4|
|   3|1001| f1-9|null| null|
|   3|1002|f1-10|1002| f2-5|
|   3|1003|f1-11|null| null|
|   3|1004|f1-12|1004| f2-6|
|   4|1001|f1-13|null| null|
|   4|1002|f1-14|1002| f2-7|
|   4|1003|f1-15|null| null|
|   4|1004|f1-16|1004| f2-8|
|   5|1001|f1-17|null| null|
|   5|1002|f1-18|1002| f2-9|
|   5|1003|f1-19|null| null|
|   5|1004|f1-20|1004|f2-10|
+----+----+-----+----+-----+

In [59]:
from pyspark.sql import functions as F

grouped_rdd = joined.groupBy(joined.id_1).agg(F.collect_list(joined.ts_1), F.collect_list(joined.f1), F.collect_list(joined.f2), F.collect_list(joined.ts))
grouped_rdd.collect()

[Row(id_1=1, collect_list(ts_1)=[1001, 1002, 1003, 1004], collect_list(f1)=['f1-1', 'f1-2', 'f1-3', 'f1-4'], collect_list(f2)=['f2-1', 'f2-2'], collect_list(ts)=[1002, 1004]), Row(id_1=3, collect_list(ts_1)=[1001, 1002, 1003, 1004], collect_list(f1)=['f1-9', 'f1-10', 'f1-11', 'f1-12'], collect_list(f2)=['f2-5', 'f2-6'], collect_list(ts)=[1002, 1004]), Row(id_1=5, collect_list(ts_1)=[1001, 1002, 1003, 1004], collect_list(f1)=['f1-17', 'f1-18', 'f1-19', 'f1-20'], collect_list(f2)=['f2-9', 'f2-10'], collect_list(ts)=[1002, 1004]), Row(id_1=4, collect_list(ts_1)=[1001, 1002, 1003, 1004], collect_list(f1)=['f1-13', 'f1-14', 'f1-15', 'f1-16'], collect_list(f2)=['f2-7', 'f2-8'], collect_list(ts)=[1002, 1004]), Row(id_1=2, collect_list(ts_1)=[1001, 1002, 1003, 1004], collect_list(f1)=['f1-5', 'f1-6', 'f1-7', 'f1-8'], collect_list(f2)=['f2-3', 'f2-4'], collect_list(ts)=[1002, 1004])]

In [ ]:
def align(ts_1, f1, f2, ts_2):
    latest_f2=0
    for i in range(len(ts_1)):
        if f2[i] != null:
            latest_f2=f2
        

In [10]:
maprdd = fg2.rdd.groupBy(lambda x:x[0]).map(lambda x:(x[0],{y[1] for y in x[1]}))
result_dict = dict(maprdd.collect()) 
result_dict

{1: {1611505152, 1611505169, 1611505154, 1611505155}, 2: {1611505154, 1611505162, 1611505159}, 3: {1611505163}, 4: {1611505165}}

In [11]:
map_broadcast = sc.broadcast(result_dict)

In [32]:
def take_closest(id, ts):
    return min(map_broadcast.value[id],key=lambda x:((ts-x) if ts >= x else sys.maxsize))
columns = ["id2","join_ts","ts2"]

my_rdd = fg1.rdd.map(lambda x: (x[0],x[1],take_closest(x[0],x[1])))
filtered_rdd = my_rdd.filter(lambda x: x[1] >= x[2])
join2=spark.createDataFrame(filtered_rdd,columns).sort(col("id2"),col("join_ts"))

# columns_to_drop = ['ts']
# join2 = join2.drop(*columns_to_drop)
join2.show()

+---+----------+----------+
|id2|   join_ts|       ts2|
+---+----------+----------+
|  1|1611505155|1611505155|
|  1|1611505156|1611505155|
|  2|1611505157|1611505154|
|  2|1611505161|1611505159|
+---+----------+----------+

In [36]:
joined = fg1.join(join2,(fg1.id==join2.id2) & (fg1.ts==join2.join_ts),how="inner")
drop_cols = ['id2', 'join_ts']
inter = joined.drop(*drop_cols)
inter.show(15)

+---+----------+---+----------+
| id|        ts| f1|       ts2|
+---+----------+---+----------+
|  1|1611505155|f11|1611505155|
|  1|1611505156|f12|1611505155|
|  2|1611505157|f21|1611505154|
|  2|1611505161|f22|1611505159|
+---+----------+---+----------+

In [44]:
inter = inter.alias('inter')
fg2 = fg2.alias('fg2')
final = inter.join(fg2,(inter.id==fg2.id) & (inter.ts2==fg2.ts),how="inner").select('inter.id', 'inter.ts', 'inter.f1', 'fg2.f2')
final.show(15)

+---+----------+---+----+
| id|        ts| f1|  f2|
+---+----------+---+----+
|  1|1611505155|f11|f213|
|  1|1611505156|f12|f213|
|  2|1611505157|f21|f220|
|  2|1611505161|f22|f221|
+---+----------+---+----+